# Imports

In [199]:
#from elasticsearch_dsl import Search, Q, Range, connections
#from elasticsearch.client import Elasticsearch
from tqdm.notebook import tqdm
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
import re
from openai import OpenAI

In [200]:
DATA_FOLDER = "../data/events/browse_ai/" 

In [201]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 20)
pd.set_option('display.max_colwidth', 300)

# Browse AI

In [202]:
try: 
    df =  pd.read_pickle(DATA_FOLDER+"df_browse_ai original DD FB.pkl")
except Exception as e:
    df = pd.read_excel(DATA_FOLDER+"! Browse.AI master file Germany.xlsx", sheet_name = "Event list") 

In [ ]:
df.columns

In [204]:
df = df.dropna(how="all", axis=1)

In [ ]:
df.columns

In [206]:
try: 
    df = df.drop(columns=['Event links Limit', 'Position', 'Event list Limit','Date updated by Zapier', 'Unnamed: 53'])
except Exception as e:
    pass

In [207]:
df = df[df.apply(lambda row: row.astype(str).str.contains('Dresden|Freiburg', case=False, na=False).any(), axis=1)]

In [208]:
df = df.dropna(how="all", axis=1)

In [ ]:
df.columns

In [ ]:
df['Location ZIP'].unique().tolist()

In [211]:
df = df[~df['Location ZIP'].isin([77977, 72250,18311,24937,30161, 71063, 95448,95100, 65428, 17192, '10245 Berlin'])]

In [212]:
#sorted(df_browse_ai['Location ZIP'].unique().tolist())

In [213]:
exclude_columns= ['Date (GMT)', 'Job Link']

subset_columns = [col for col in df.columns if col not in exclude_columns]

df = df.drop_duplicates(subset=subset_columns)

In [214]:
df = df [~df["Location address"].isin(['GRÜNWALDER STADION\nAdresse: Grünwalder Str. 4, 81547 München', 
                                       'Centralkomitee\nSteindamm 45\n20099 Hamburg\nGoogle Maps'])]

In [215]:
df = df [~df["Start date"].isin([])]

## exclude: 4,5,6,7,8, 11, 12 inclusive

In [216]:
df['Start date'] = df['Start date'].astype(str)

In [217]:
df['Start date'] = df['Start date'].astype(str)

In [ ]:
df.columns

In [219]:
date_columns = ['Start date and time', 'Start time', 'Start date and end date', 'Start date']

In [220]:
for weekday in ['Dienstag, ', "Donnerstag, ", "Freitag, ", "Mittwoch, ", "Montag, ", "Samstag, ", "Sonntag, ", "Sonntag ", "Mittwoch ", "Samstag "]:
    df[date_columns] = df[date_columns].apply(lambda col: col.str.replace(weekday, '', regex=True))

In [221]:
df['Start date']= df['Start date'].str.replace(" September", '09.2024')

In [222]:
for col in date_columns:
    df[col] = df[col].astype(str)

In [223]:
df['Start date and end date']= df['Start date and end date'].astype(str)

In [ ]:
len(df)

In [225]:
months_to_exclude = [4,5,6,7,8,11, 12]

In [226]:
for i, row in df.iterrows():
    try:
        date = pd.to_datetime(row["Start date"],format = "%Y-%m-%d")
        if date.month in months_to_exclude:
            df =  df.drop(i)
    except Exception as e:
        pass
        

In [227]:
for i, row in df.iterrows():
    try:
        date = pd.to_datetime(row["Start date"],format = "%d.%m.%Y")
        if date.month in months_to_exclude:
            df =  df.drop(i)
    except Exception as e:
        pass
        

In [ ]:
for i, row in df.iterrows():
    try:
        dt_str = row["Start date and end date"][:10]
        print(dt_str)
        date = pd.to_datetime(dt_str,format = "%d.%m.%Y")
        if date.month in months_to_exclude:
            df =  df.drop(i)
    except Exception as e:
        pass

    try:
        dt_str = row["Start date and end date"][:-10]
        print(dt_str)
        date = pd.to_datetime(dt_str,format = "%d.%m.%Y")
        if date.month in months_to_exclude:
            df =  df.drop(i)
    except Exception as e:
        pass
        

In [ ]:
len(df)

In [230]:
#sorted(df_browse_ai['Start date'].unique().tolist())

In [ ]:
date_columns

In [ ]:
sorted(df['Start time'].astype(str).unique().tolist())

In [ ]:
sorted(df['Start date and time'].astype(str).unique().tolist())

In [234]:
df = df[~df['Start date and time'].str.contains('02.12.2024 19:30 Uhr', na=False)]

In [235]:
#sorted(df_browse_ai['Start date and end date'].astype(str).unique().tolist())

In [236]:
parts_to_drop = ["until\xa0Sun, May 25, 2025, 08:00 PM", "Dez ", "Nov ", "2025", "abgesagt", 
                 "09.08.2024 bis 10.08.2024", "04.07.2024 bis 09.07.2024", "06.07.2024 bis 28.07.2024", "07.12.2024", "17.05.2024 bis 27.05.2024"]
pattern = "|".join(parts_to_drop)


In [237]:
df = df[~df['Start date and end date'].str.contains(pattern, na=False, case=True)]

In [238]:
df = df[~df['Start date and end date'].str.contains("Der nächste Termin ist uns noch nicht bekannt. ", na=False)]

In [ ]:
len(df)

In [240]:
#sorted(df_browse_ai['Location city'].astype(str).unique().tolist())

In [241]:
# keep Heidenau  Radebeul Am Schwabentor Bad Krozingen Bannewitz (На всяк випадок)

In [242]:
city_list = ["Kamenz", "Leipzig", "Freital", "Zwickau", "Radeberg", "Meißen", "Oschatz", "Pulsnitz", "Liechtenstein", "Bad Elster", "Bad Schandau", "Cunewalde",
"Königstein", "Gaienhofen", "Heidenheim an der Brenz", "Hinterzarten", "Denzlingen", "Reutlingen", "Kriebstein", "Leisnig", "Chemnitz",
             "Donaueschingen",
"Marienberg", "Schönbach", "Singen (Hohentwiel)", "Stockach", "Tharandt", "Weesenstein", "Großröhrsdorf", "Grimma-Döben", "Lichtenstein", "Schorndorf", "Hoyerswerda"
            
            ]

In [243]:
df = df[~df['Location city'].isin(city_list)]

In [ ]:
pattern = '|'.join(city_list)
df = df[~df['Location address'].str.contains(pattern,case=True, na=False, regex=True)]

In [ ]:
len(df)

In [246]:
manually_excluded_ids = [2985, 2986, 16125, 3273, 3291]

In [247]:
df =  df.drop(manually_excluded_ids, errors = "ignore")

In [ ]:
len(df)

In [249]:
# Exclude complicated cases for now
#df_browse_ai = df_browse_ai[~(df_browse_ai[date_columns]=="nan").all(axis=1)]

In [ ]:
len(df)

# Extract info

In [251]:
correct_openai_ids=[]
incorrect_openai_ids=[]

## extract dates

In [252]:
for colname in  ["extrStartDate", "extrStartTime","extrEndTime" , "isOnMultipleDays","extrStartDateMultiple","extrEndDateMultiple", "extrMultipleDaysDetails"]:
    df[colname]="nan"

In [253]:
extended_date_columns = ['Start date and time', 'Start time', 'Start date and end date', 'Start date', "extrStartDate", "extrStartTime", "extrEndTime", "isOnMultipleDays","extrStartDateMultiple","extrEndDateMultiple", "extrMultipleDaysDetails"]

### multiple days

In [254]:
id = 3064
df.loc[id, "isOnMultipleDays"]=True
df.loc[id, "extrStartDateMultiple"] = "2024-06-08"
df.loc[id, "extrStartDateMultiple"] = "2024-09-10"
df.loc[id, "extrMultipleDaysDetails"] = """Wed-Fri: 10:00-17:00, Sat-Sun: 12:00-17:00"""
incorrect_openai_ids.append(id)

In [255]:
id = 3052
df.loc[id, "isOnMultipleDays"]=True
df.loc[id, "extrStartDateMultiple"] = "2024-03-24"
df.loc[id, "extrStartDateMultiple"] = "2024-10-06"
df.loc[id, "extrMultipleDaysDetails"] = """FAHRSAISON:\n\n24. März 2024\nEröffnung der Saison\nRundfahrten von 13:00 bis 18:00 Uhr\n\n27. März bis 6. Oktober\xa02024\nMittwoch bis Freitag: 13:00 bis 18:00 Uhr\nSamstag, Sonntag, Feiertag: 10:30 bis 18:00 Uhr\n\n9. bis 26. Oktober 2024\nMittwoch bis Freitag: 13:00 bis 17:00 Uhr\nSamstag, Sonntag, Feiertag: 10:30 bis 17:00 Uhr\n\nKein Fahrbetrieb am 9. Mai (Himmelfahrt) """
incorrect_openai_ids.append(id)

In [256]:
id =2289 
df.loc[id, "isOnMultipleDays"]=True
df.loc[id, "extrStartDateMultiple"]="2024-09-14"
df.loc[id, "extrEndDateMultiple"]="2024-09-22"
df.loc[id, "extrEndDateMultiple"] = """Ein Ungarischer Markt (18 bis 22. September 2024), der Dresdner\xa0Pilztag (15. September 2024) sowie der\xa0Brotmarkt der Bäcker-Innung Dresden (14. und 15. September 2024)
"""
incorrect_openai_ids.append(id)

In [257]:
id = 3262
df.loc[id, "isOnMultipleDays"]=True
df.loc[id, "extrStartDateMultiple"]="2024-09-01"
df.loc[id, "extrEndDateMultiple"]="2024-10-13"
df.loc[id, "extrMultipleDaysDetails"] = """Di - Fr 9 bis 17 Uhr, Sa/So 10 bis 18 Uhr"""
incorrect_openai_ids.append(id)

In [258]:
id = 3264 # todo change
df.loc[id, "isOnMultipleDays"]=True
df.loc[id, "extrStartDateMultiple"]="2024-09-01"
df.loc[id, "extrEndDateMultiple"]="2024-11-03"
df.loc[id, "extrMultipleDaysDetails"] = """Mi – Fr 16 – 19 Uhr, Sa & So und Feiertag 12 -18 Uhr"""
incorrect_openai_ids.append(id)
#df_browse_ai.loc[id]

In [259]:
id = 29466
df.loc[id, "isOnMultipleDays"]=True
df.loc[id, "extrStartDateMultiple"]="2024-10-18"
df.loc[id, "extrEndDateMultiple"]="2024-10-28"
df.loc[id, "extrMultipleDaysDetails"] =  """Fr. 18.10. 17:00 – 23:00 Sa. 19.10. 14:00 – 23:00 So. 20.10. 11:00 – 22:00 Mo. 21.10. 14:00 – 22:00 Di. 22.10. 14:00 - 23:00 Mi. 23.10. 14:00 – 22:00 Do. 24.10. 14:00 – 22:00 Fr. 25.10. 14:00 – 23:00 Sa. 26.10. 14:00 – 23:00 So. 27.10. 11:00 – 22:00 Mo. 28.10. 14:00 – 23:00"""
incorrect_openai_ids.append(id)

In [260]:
id = 10469
df.loc[id, "isOnMultipleDays"]=True
df.loc[id, "extrStartDateMultiple"]="2024-09-26"
df.loc[id, "extrEndDateMultiple"]="2024-09-28"
df.loc[id, "extrMultipleDaysDetails"] = "assume time 12 - 21"
incorrect_openai_ids.append(id)

In [261]:
df.loc [df["Start date and end date"].str.contains("bis"), "isOnMultipleDays"]=True

In [262]:
row_flt = (df.isOnMultipleDays==True) & (df.extrStartDateMultiple=="nan")
df.loc[row_flt, "extrStartDateMultiple"] = df.loc[row_flt, "Start date and end date"].str[:10]
df.loc[row_flt, "extrEndDateMultiple"] = df.loc[row_flt, "Start date and end date"].str[-10:]

In [263]:
#df_browse_ai[row_flt]

In [ ]:
df.columns

In [265]:
id = 614     
df.loc[id]
df.loc[id, "extrMultipleDaysDetails"] = """Eröffnungstag von 17 bis 23 Uhr
Samstag von 14 bis 23 Uhr
Sonntag von 11 bis 22 Uhr
Montag von 14 bis 22 Uhr
Dienstag von 14 bis 23 Uhr
Mittwoch von 14 bis 22 Uhr
Donnerstag von 14 bis 22 Uhr
Freitag von 14 bis 23 Uhr
Abschlusstag von 14 bis 23 Uhr"""
incorrect_openai_ids.append(id)

In [266]:
id = 2275  
df.loc[id, "extrMultipleDaysDetails"] = """Donnerstag von 11 bis 21 Uhr
Freitag von 15 bis 21 Uhr
Samstag von 11 bis 21 Uhr
Sonntag von 11 bis 19 Uhr"""
incorrect_openai_ids.append(id)

In [267]:
id = 2289             
df.loc[id]
df.loc[id, "extrMultipleDaysDetails"] = """
Eröffnungstag von 12 bis 20 Uhr
Montag bis Donnerstag von 10 bis 19 Uhr
Freitag und Samstag von 10 bis 20 Uhr
Sonntag von 10 bis 19 Uhr
Tag der Deutschen Einheit von 10 bis 20 Uhr"""
incorrect_openai_ids.append(id)

In [268]:
id = 2291                 
df.loc[id, "extrMultipleDaysDetails"] = """Samstag und Sonntag von 10 bis 18 Uhr"""
incorrect_openai_ids.append(id)

In [269]:
id = 2297                     
df.loc[id]
df.loc[id, "extrMultipleDaysDetails"] =  """Freitag von 14 bis ca. 21 Uhr Samstag von 10 bis ca. 21 Uhr Sonntag von 11 bis ca. 18 Uhr"""
incorrect_openai_ids.append(id)

### single days

In [ ]:
id = 3257  
df.loc[id, "isOnMultipleDays"] = False
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrStartTime"] = "16"
df.loc[id, "extrEndTime"] = "20"
df.loc[id]

In [271]:



df.loc[id, "extrStartDate"] = "2024-09-25"
df.loc[id, "extrStartTime"] = "20"
incorrect_openai_ids.append(id)


In [272]:
id = 10430  
df.loc[id, "isOnMultipleDays"] = False
df.loc[id, "extrStartDate"] = "2024-09-25"
df.loc[id, "extrStartTime"] = "20"
incorrect_openai_ids.append(id)



In [273]:
id = 10468
df.loc[id, "isOnMultipleDays"] = False
df.loc[id, "extrStartDate"] = "2024-09-28"
df.loc[id, "extrStartTime"] = "20"
incorrect_openai_ids.append(id)

In [ ]:
df.isOnMultipleDays.value_counts()

In [275]:
flt_rows = (df.isOnMultipleDays!=True) &  (df["Start date"]!="nan")
df.loc[:, "Start date"] = df.loc[flt_rows, "Start date"].str.replace(" 00:00:00", "")

In [ ]:
len(df.loc[flt_rows])

In [ ]:
df.loc[flt_rows, "Start date"].unique()

In [ ]:
flt_rows = (df.isOnMultipleDays!=True) & (df["Start date"].str.contains(".", regex=False))
df.loc[flt_rows, "Start date"].unique()

In [279]:

df.loc[flt_rows, "extrStartDate"] = pd.to_datetime(df.loc[flt_rows, "Start date"], errors="coerce", format="%d.%m.%Y").dt.strftime("%Y-%m-%d") 
df.loc[flt_rows, "isOnMultipleDays"]=False

In [ ]:
flt_rows = (df.isOnMultipleDays!=True) & (df.extrStartDate=="nan") & (df["Start date"].str.contains("-", regex=False))
df.loc[flt_rows, "Start date"].unique()


In [281]:
df.loc[flt_rows, "extrStartDate"] = pd.to_datetime(df.loc[flt_rows, "Start date"], errors="coerce", format="%Y-%m-%d").dt.strftime("%Y-%m-%d")
df.loc[flt_rows, "isOnMultipleDays"]=False

In [ ]:
df.extrStartDate.unique()

In [ ]:
df.isOnMultipleDays.value_counts()

In [284]:
col = df["Start date and end date"]
col = col.astype(str)

In [ ]:
df.columns

In [286]:
flt_rows = (df.isOnMultipleDays=="nan") & (df["Start date and end date"].str[-3:]=="Uhr")
df.loc[flt_rows, "extrStartTime"] = df.loc[flt_rows, "Start date and end date"].str[-9:-4].str.replace(":30", ",5").str.replace(":00", "")
df.loc[flt_rows, "extrStartDate"] = pd.to_datetime(df.loc[flt_rows, "Start date and end date"].str[19:-10].str.replace(" Sep ", "09.").str.replace(" Okt ", "10."), format="%d.%m.%Y").dt.strftime("%Y-%m-%d")
df.loc[flt_rows, "isOnMultipleDays"] = False


In [287]:
def change_PM_hour_minute_to_my_format(s):
    spl = s.split(":")
    hour= int(spl[0])
    if hour<12:
        hour=hour+12
    hour=str(hour)
    if spl[1]=="15":
        minute=""
    elif spl[1]=="30":
        minute=",5"
    else:
        minute=""

    return "".join([hour, minute])
        

In [288]:
pattern = r"\b\d{1,2}:\d{2}\sPM\sTime\b"
flt_rows = (df.extrStartDate!="nan") &  (df.extrStartTime=="nan") \
& (df["Start date and end date"]!="nan" ) & (df["Start date and end date"].str.contains(pattern))
values = df.loc[flt_rows, "Start date and end date"].str.extract(r"(\d{1,2}:\d{2})")[0].apply(change_PM_hour_minute_to_my_format)
df.loc[flt_rows, "extrStartTime"] = values

In [ ]:
flt_rows = (df.extrStartTime=="nan") & (df["Start time"]!="nan")
df.loc[flt_rows]

In [290]:
df.loc[3234, "extrStartTime"]="20"
df.loc[3234, "extrEndTime"]="22"
df.loc[18673, "extrStartTime"]="20"
df.loc[18673, "extrEndTime"]="21"
incorrect_openai_ids.append(3234)
incorrect_openai_ids.append(18673)

In [291]:
def change_start_date_and_time_to_my_format(s):
    spl = s.split(" ")
    hour_minute=spl[1]
    return change_PM_hour_minute_to_my_format(hour_minute)

In [292]:
flt_rows =(df.extrStartDate!="nan") &  (df.extrStartTime=="nan") &  (df["Start date and time"]!="nan")
values = df.loc[flt_rows]["Start date and time"].apply(change_start_date_and_time_to_my_format) 
df.loc[flt_rows,"extrStartTime"]=values

In [293]:
df.loc[4242, "extrStartTime"]="10,5"
df.loc[4558, "extrStartTime"]="10,5"
incorrect_openai_ids.append(4242)
incorrect_openai_ids.append(4558)

### remaining observations

In [ ]:
extended_date_columns

In [295]:
#df_browse_ai [(df_browse_ai.extrStartDate!="nan") &  (df_browse_ai.extrStartTime=="nan") &  (df_browse_ai.extrMultipleDaysDetails=="nan")][["Event description"]]

In [296]:
# df_browse_ai [df_browse_ai.isOnMultipleDays=="nan"][["Event description"]]

## extract addresses

### extrCity

In [ ]:
df.columns

In [ ]:
df["Location city"]

In [299]:
df["extrCity"] = df["Location city"].astype(str)

In [ ]:
df.extrCity

In [301]:
df.extrCity = df.extrCity.str.replace("Freiburg im Breisgau", "Freiburg")
df.extrCity = df.extrCity.str.replace("Am Schwabentor", "Freiburg")
df.extrCity = df.extrCity.str.replace("Radebeul", "Dresden")
df.extrCity = df.extrCity.str.replace("Heidenau", "Dresden")
df.extrCity = df.extrCity.str.replace("Bannewitz", "Dresden")

In [ ]:
df.extrCity.unique()

In [303]:
flt = df.extrCity.str.contains("Dresden")
df.loc[flt, "extrCity"]="Dresden"
flt = df.extrCity.str.contains("Freiburg")
df.loc[flt, "extrCity"]="Freiburg"

In [ ]:
df.groupby("extrCity").size()

In [305]:
df.columns = [col.replace(' ', '_') for col in df.columns]

In [306]:
for city_name in ["Dresden", "Freiburg"]:
    for colname in ["Event_name"]:
        flt = (df[colname].str.contains(city_name)) & (df.extrCity=="nan")
        df.loc[flt, "extrCity"] = city_name
        

In [307]:
location_columns = [col for col in df.columns if col.startswith("Location")]

In [308]:
for city_name in ["Dresden", "Freiburg"]:
    for colname in location_columns:
        flt = (df[colname].astype(str).str.contains(city_name)) & (df.extrCity=="nan")
        df.loc[flt, "extrCity"] = city_name
        

In [309]:
exclude_ids = [3886, 6476, 10146, 11081, 27519, 21798, 24265, 29848, 30460, 30501, 30856, 23605, 5581, 19882, 24122, 18673,3234, 24209] # exclude because of other city
df.drop(exclude_ids, inplace=True, errors = "ignore")

### extrStreetHouseNumber

In [310]:
df["extrStreetHouseNumber"] = "nan"

In [311]:
df["extrLocationDetails"] = "nan"

In [312]:
id = 25851
df.loc[id, "isOnMultipleDays"]=False
df.loc[id, "extrCity"]="Freiburg"
df.loc[id, "extrStartDate"]="2024-10-25"
df.loc[id, "extrStartTime"] = "18"
df.loc[id, "extrStreetHouseNumber"] = "Franziskanerstrasse 11"
incorrect_openai_ids.append(id)


In [313]:
id = 25832
df.loc[id, "isOnMultipleDays"]=False
df.loc[id, "extrCity"]="Freiburg"
df.loc[id, "extrStartDate"]="2024-10-25"
df.loc[id, "extrStartTime"] = "19"
df.loc[id, "extrStreetHouseNumber"] = "Grünwälderstrasse 6"
incorrect_openai_ids.append(id)

In [314]:
df.loc[flt, "extrStreetHouseNumber"] = df.loc[flt, "Location_address"]
df.extrStreetHouseNumber =df.extrStreetHouseNumber.astype(str) 

In [315]:
df.extrStreetHouseNumber = df.extrStreetHouseNumber.str.replace("Dresden", "")
df.extrStreetHouseNumber = df.extrStreetHouseNumber.str.replace("Freiburg im Breisgau", "")
df.extrStreetHouseNumber = df.extrStreetHouseNumber.str.replace("\n", " ")

In [316]:
postal_code_pattern = r'\b\d{5}\b'

In [317]:
df.extrStreetHouseNumber = df.extrStreetHouseNumber.str.replace(postal_code_pattern, "", regex=True).str.strip()

In [318]:
flt = (df.extrStreetHouseNumber=="nan") & (df.Organizer_details.str.contains("Veranstaltungsort"))
df.loc[flt, "extrLocationDetails"] = df.loc[flt, "Organizer_details"]

In [319]:
#df_browse_ai.loc[flt, "extrLocationDetails"].str.replace("\n", " NNN ")

In [320]:
def extract_street_hn_from_org_details(s: str):
    ls  = s.split("\n")
    street_hn = ls[2]
    street_hn = re.sub(postal_code_pattern, "", street_hn).strip()
    return street_hn

In [321]:
flt = (df.extrLocationDetails!="nan") 
values = df.extrLocationDetails[flt].apply(extract_street_hn_from_org_details)
df.loc[flt, "extrStreetHouseNumber"]=values

In [322]:
all_cols_except_descr = df.columns.tolist()

In [323]:
all_cols_except_descr.remove("Event_description")

In [324]:
flt = (df.extrStreetHouseNumber=="nan") & (~df.Location_details.isna())
values = df.loc[flt,"Location_address"]
df.loc[flt, "extrStreetHouseNumber"] = values


In [325]:
flt = (df.extrStreetHouseNumber=="nan") & (~df.Location_address.isna()) & (df.Origin_URL.str.contains("https://www.reservix.de/veranstaltungskalender?page=12", regex=False))
values = df.loc[flt, "Location_address"]
df.loc[flt, "extrStreetHouseNumber"] = values

In [326]:
flt = (df.extrStreetHouseNumber=="nan") & (df.Location_name.isna())
values = df.loc[flt, "Location_address"]
df.loc[flt, "extrLocationDetails"]=values

### remaining observations

In [ ]:
df.columns

In [ ]:
df.extrLocationDetails.drop_duplicates()

In [ ]:
flt = (df.extrStreetHouseNumber=="nan") & (~df.Location_address.isna())
df.loc[flt, ["Origin_URL", "Location_address"]]

In [330]:
df.loc[:, "count_Event_description"]= df.Event_description.map(df.Event_description.value_counts())

In [ ]:
df.columns

In [ ]:
flt = (df.count_Event_description>1) # &  (df.extrMultipleDaysDetails!="nan") 
len(df[flt])

In [333]:
flt = (df.extrStreetHouseNumber=="nan") & (df.Location_name.isna())
#df.loc[flt, location_columns]

## extract time

## Extract start time with OpenAI

In [334]:
df["OpenAIResponse"] = "nan"


In [335]:
#df_browse_ai [df_browse_ai.isOnMultipleDays=="nan"][["Event_description"]]

In [ ]:
flt = df.Event_description.str.contains("Uhr", na=False)
len(df[flt][["Event_description"]])

with open("OpenAIKey (vkalonova).txt", "r") as file:
        openai_api_key = file.read().strip()

client = OpenAI(api_key=openai_api_key)

system_content = """
I give you a text. Print for me the extracted details about the event date in the following example format:
Example input 1:

Example response 1:
Is on multiple days: no
Start date: 30.09.2024
End date: NA
Start time: 10:30
End time: 18:45
MultipleDaysDetails: NA

Example input 2:

Example response 2:
Is on multiple days: yes
Start date: 30.09.2024
End date: 02.10.2024
Start time: NA
End time: NA
MultipleDaysDetails: Fri: 09:00-18:00, Sat: 12:00-18:00, Sun: 14:00-15:30

Example input 3:
Der Keramikmarkt in Dresden findet 2024 bereits zum 29. Mal statt. Rund um den Goldenen Reiter präsentieren etwa 70 Aussteller und Ausstellerinnen ihre Exponate. Regionale Keramiker sind genauso vertreten wie anerkannte Künstler aus Nah und Fern. Die Palette reicht von von zeitgenössischem Schmuck über bedrucktes Steinzeug bis hin zu Gartenkeramik und Skulpturen. Egal ob Einzelstück oder serielle Arbeiten, ob schlichte Formen oder üppige Dekore - hier gibt es für alle Geschmäcker etwas Passendes. Der Keramikmarkt in Dresden wird musikalisch Folk, Jazz, Tango und Musette von "Krambambuli" untermalt. Für die Kinder gibt es wieder ein Puppenspiel. Der Keramikmarkt in Dresden wird in der Regel am ersten Septemberwochenende abgehalten. (© FuM)'
Example response 3:


It the text does not contain the information, do not invent anything, but fill the missing values with "NA".
"""

prompt = """
Wer entscheidet eigentlich, wer dazugehört? Beheimatet zu sein, Teil einer Gesellschaft zu sein, und „dazuzugehören“ ist eine alltägliche aktive Handlung. Wir gehören dazu, weil wir uns verorten und Teil dieser Gesellschaft sind, als Menschen in all unserer Diversität, mit all unseren Herkunftsgeschichten, besonderen Begabungen und individuellen Formen des Zusammenlebens und der Nähe. Doch was bedeutet das, in Zeiten, in denen Solidarität, Menschlichkeit und Selbstbestimmung über den eigenen Körper und eigene Lebensentwürfe von Teilen der Gesellschaft in Frage gestellt werden?\n\nMit den Künstler*innen: Thomas Bratzke, Lysann Buschbeck, Talya Feldman, Kanak Attak, Amac Garbe, Raisan Hameed, Taemen Jung, Susanne Keichel, Irène Mélix, Minh Duc Pham, Joe Richter & Florian Magnus Puls, Maya Schweizer, Anja Schneider, Anna Shapiro, Su-Ran Sichling, Alina Simmelbauer, Antje Bajer & André Tempel, Tribunal Spots, Shira Wachsmann, Claire Waffel, Ina Wudtke, Nazanin Zandi & Queer Deities in Migration & die Blaue Distanz (Adam Erdmann & Franzi Goralski)\n\nGefördert durch die Kulturstiftung des Freistaates Sachsen und den Stadtbezirksbeirat Dresden-Altstadt. Lila-x-tage x Queer Deities in Migration is co-funded by the Cultural Foundation of Saxony and the Creative Europe programme of the European Union.\nMit freundlicher Genehmigung durch Gateway Real Estate.\n\nAusstellungseröffnung 30.08.2024 um 17:00 Uhr\n\nDie Ausstellung findet in der robotron-Kantine statt, dem neuen Sitz des Kunsthauses Dresden.\n\nÖffnungszeiten\nMi – Fr 16 – 19 Uhr\nSa & So und Feiertag 12 -18 Uhr\nEintritt frei\n\nhttps://kunsthausdresden.de/b
"""

""" completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": system_content},
        {
            "role": "user",
            "content": prompt
        }
    ]
)

print(completion.choices[0].message) """

In [337]:
#print(completion.choices[0].message.content)

In [338]:
#completion

In [ ]:
flt = (df.isOnMultipleDays=="nan") & (~ df.Event_description.isna())
len(df[flt])

In [ ]:
""" flt = (~df_browse_ai.Event_description.isna()) & (df_browse_ai.OpenAIResponse=="nan")

for i, row in tqdm(df_browse_ai[flt].iterrows()):
    print(i)
    prompt= row["Event_description"]
    completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": system_content},
        {
            "role": "user",
            "content": prompt
        }
    ]
    )
    
    df_browse_ai.loc[i, "OpenAIResponse"]=completion.choices[0].message.content
    
    
     """

In [ ]:
df.columns

In [342]:
flt = (df.extrStartDateMultiple!="nan") & (df.extrStartDate=="nan")
values = df.loc[flt, "extrStartDateMultiple"]
df.loc[flt, "extrStartDate"]=values
df.drop(columns=["extrStartDateMultiple"], inplace=True)

In [343]:
# rename 1 column in a df
df.rename(columns = {"extrEndDateMultiple":"extrEndDate"}, inplace=True)

In [ ]:
df.columns

In [345]:
def transform_openai_date(s:str):
    if s=="NA":
        transformed_date = "nan"  
    else:
        try:    
            transformed_date = pd.to_datetime(s, format="%d.%m.%Y").strftime("%Y-%m-%d") 
        except Exception as e:
            transformed_date = s
    return transformed_date

In [346]:
flt = df.extrStartDate.str.contains(".")
df.loc[flt, "extrStartDate"] = df.loc[flt, "extrStartDate"].apply(transform_openai_date)

In [347]:
flt = df.extrEndDate.str.contains(".")
df.loc[flt, "extrEndDate"] = df.loc[flt, "extrEndDate"].apply(transform_openai_date)

In [348]:
df_with_openai_responses = pd.read_pickle(DATA_FOLDER+"df_browse_ai with openai responses 2024-12-10_12-37-15.pkl")

In [349]:
df.OpenAIResponse = df_with_openai_responses.OpenAIResponse

In [350]:
date_columns = [col.replace(" ", "_") for col in   date_columns]

In [351]:

correct_openai_ids.extend([14601, 15083 , 15559, 15615, 16071, 16093    ])

In [352]:
correct_openai_ids.extend([14601, 15083 , 15559, 15615, 16071, 16093, 3029, 3031, 3032, 3035, 3036 , 3037, 3038, 3039 , 3049    ])
incorrect_openai_ids.extend([14630, 14650, 14655, 15590, 15610 , 16125,2289])

### structure openai responses

In [ ]:
df.loc[3052                         , "OpenAIResponse"]

In [ ]:
df.columns

In [355]:
for colname in ["OpenAIResponse_isOnMultipleDays", "OpenAIResponse_StartDate", \
                "OpenAIResponse_EndDate", "OpenAIResponse_StartTime", "OpenAIResponse_EndTime", "OpenAIResponse_MultipleDaysDetails"]:
    df[colname]="nan"

In [356]:
flt = (df.OpenAIResponse!="nan")

for i, row in df[flt].iterrows():
    response = row["OpenAIResponse"]
    event_description = row['Event_description']
    #print(f"{response=}")
    #print(f"{row['Event_description']=}")

    responses = response.split("\n")
    isOnMultipleDays = responses[0].replace("Is on multiple days: ", "").strip()
    if isOnMultipleDays=="yes":
        isOnMultipleDays=True
    elif isOnMultipleDays=="no":
        isOnMultipleDays=False
    else:
        isOnMultipleDays="nan"
    
    extrStartDate = responses[1].replace("Start date: ", "").strip()
    extrStartDate = transform_openai_date(extrStartDate)
    year = extrStartDate[:4]
    if year not in ["2023", "2024", "2025"]:
        extrStartDate = "nan"

    extrEndDate = responses[2].replace("End date: ", "").strip()
    extrEndDate = transform_openai_date(extrEndDate)


    extrStartTime = responses[3].replace("Start time: ", "").strip()
    extrStartTime="nan" if extrStartTime=="NA" else extrStartTime

    extrEndTime = responses[4].replace("End time: ", "").strip()
    extrEndTime="nan" if extrEndTime=="NA" else extrEndTime

    if i == 15615:
        extrEndDate = "2024-10-31"
    if "verlängert bis Frühjahr 2025" in event_description:
        extrEndDate = "2024-10-31"
        isOnMultipleDays = True


    extrMultipleDaysDetails = responses[5].replace("MultipleDaysDetails: ", "").strip()
    extrMultipleDaysDetails="nan" if extrMultipleDaysDetails=="NA" else extrMultipleDaysDetails

    df.loc[i, "OpenAIResponse_isOnMultipleDays"]=isOnMultipleDays
    df.loc[i, "OpenAIResponse_StartDate"]=extrStartDate
    df.loc[i, "OpenAIResponse_EndDate"]=extrEndDate
    df.loc[i, "OpenAIResponse_StartTime"]=extrStartTime
    df.loc[i, "OpenAIResponse_EndTime"]=extrEndTime
    df.loc[i, "OpenAIResponse_MultipleDaysDetails"]=extrMultipleDaysDetails    

### isOnMultipleDays

In [357]:
# believe openai on isOnMultipleDays when he says it is on multiple days
#df_browse_ai.groupby(["isOnMultipleDays", "OpenAIResponse_isOnMultipleDays"]).size()
#df_browse_ai.groupby(["Origin_URL", "isOnMultipleDays", "OpenAIResponse_isOnMultipleDays"]).size()
#flt = (df_browse_ai.isOnMultipleDays==False) & (df_browse_ai.OpenAIResponse_isOnMultipleDays==True)
#df_browse_ai.loc[flt]

In [358]:
flt = df.OpenAIResponse_isOnMultipleDays==True
df.loc[flt, "isOnMultipleDays"]=True

In [ ]:

df.query("OpenAIResponse_isOnMultipleDays!=isOnMultipleDays  ").groupby(["isOnMultipleDays", "OpenAIResponse_isOnMultipleDays"]).size()

In [ ]:
len(df.query("isOnMultipleDays ==False & extrMultipleDaysDetails!='nan'")) # should be 0

### extrStartDate

In [ ]:
flt = (df.extrStartDate=="nan") & (df.isOnMultipleDays==False)
df[flt] # empty

### extrStartTime

In [362]:
pd.set_option('display.max_colwidth', None)

In [363]:
# openai is mostly correct on the time
#flt = (df.extrStartTime=="nan") & (df.isOnMultipleDays==False) & (df.OpenAIResponse_StartTime!="nan")
#df.loc[flt, ["Event_description", "OpenAIResponse_StartTime", "OpenAIResponse_EndTime"]] 

In [364]:
def change_OpenAI_hm_to_standard_format(s):
    spl = s.split(":")
    hour= int(spl[0])
    if len(spl)==1:
        minute=""
    else:
        if spl[1]=="15":
            minute=""
        elif spl[1]=="30":
            minute=",5"
        elif spl[1]=="45":
            minute=""
            hour+=1
        else: 
            minute=""
    return "".join([str(hour), minute])
        

In [365]:
pd.set_option("display.max_rows", None)

In [366]:
flt = (df.extrStartTime=="nan") & (df.isOnMultipleDays==False) & (df.OpenAIResponse_StartTime!="nan")
values = df.loc[flt, "OpenAIResponse_StartTime"].apply(change_OpenAI_hm_to_standard_format)
df.loc[flt, "extrStartTime"]=values

In [367]:
flt = (df.extrStartTime=="nan")& (df.isOnMultipleDays==False) & (df.Event_description.str.contains("Uhr", na=False))
values = df.loc[flt,"Event_description"].str.extract(r"(.{0,7}Uhr)")[0].str.replace("ar: ", "").str.extract(r"([\d|\:]+)")[0].apply(change_OpenAI_hm_to_standard_format)
values
df.loc[flt, "extrStartTime"]=values


In [368]:
df.loc[3156, "isOnMultipleDays"]=True

In [ ]:
df.columns

In [370]:
flt = (df.isOnMultipleDays==False) & (df.extrStartTime=="nan") 
# df[flt]

### OpenAIResponse_MultipleDaysDetails, OpenAIResponse_EndDate, OpenAIResponse_EndTime, OpenAIResponse_StartTime

In [371]:
flt = df.extrMultipleDaysDetails=="nan"
df.loc[flt, "extrMultipleDaysDetails"] = df.loc[flt, "OpenAIResponse_MultipleDaysDetails"]

In [372]:
flt = df.extrEndDate=="nan"
df.loc[flt, "extrEndDate"] = df.loc[flt, "OpenAIResponse_EndDate"]

In [ ]:
df.columns

In [374]:
flt = df.extrEndTime=="nan"
df.loc[flt, "extrEndTime"] = df.loc[flt, "OpenAIResponse_EndTime"]

In [375]:
flt = df.extrEndTime=="nan"
df.loc[flt, "extrStartTime"] = df.loc[flt, "OpenAIResponse_StartTime"]

In [ ]:
df.groupby("Event_category").size()

### imply start and end time for ausstellungen and 

In [ ]:
df.loc[flt, "extrEndTime"].value_counts()

In [ ]:
flt = (df.Event_category=="Ausstellungen & Museen")
df.loc[flt, "extrStartTime"].value_counts()

In [ ]:
df.columns

In [ ]:
df.loc[flt, "extrMultipleDaysDetails"].value_counts()

### extrLocationName

In [381]:
df["extrLocationName"]=df.Location_name.fillna("nan")

### group by same event description

In [ ]:
len(df)

In [383]:
df.to_pickle(f"{DATA_FOLDER}df_browse_ai before description {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.pkl")

# BOOKMARK

In [474]:
df = pd.read_pickle(f"{DATA_FOLDER}df_browse_ai before description 2024-12-19_18-27-21.pkl")

In [ ]:
id = max(df.index) + 1
print(id)
flt = df["Event_description"].str.contains("Caspar David Friedrich. Wo alles begann", na=False)
assert len(df[flt])>0
df = df[~flt]
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrStreetHouseNumber"]="Tzschirnerpl. 2"
df.loc[id, "extrMultipleDaysDetails"] = """Öffnungszeiten Freitag-Montag, Mittwoch 10-18 Uhr Donnerstag 10-21 Uhr Dienstag geschlossen"""


In [ ]:
id = max(df.index) + 1
print(id)
flt = df["Event_description"].str.contains("Napoleon mit seiner Grande Armée marschierte am 24.", na=False)
assert len(df[flt])>0
df = df[~flt]
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-27"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationName"]="Militärhistorisches Museum der Bundeswehr"
df.loc[id, "extrStreetHouseNumber"]="Olbrichtpl. 2"
df.loc[id, "extrLocationDetails"]="Olbrichtpl. 2, 01099 Dresden"
df.loc[id, "extrMultipleDaysDetails"] = """Montag 10 - 21 Uhr Dienstag, Donnerstag, Freitag, Samstag, Sonntag 10-18 Uhr Mittwoch geschlossen"""


In [ ]:
id = max(df.index) + 1
print(id)
flt = df["Event_description"].str.contains("Wer entscheidet eigentlich, wer dazugehört", na=False)
assert len(df[flt])>0
df = df[~flt]
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationName"]="Robotron-Kantine in Dresden"
df.loc[id, "extrStreetHouseNumber"]="Zinzendorfstraße 5"
df.loc[id, "extrLocationDetails"]="Zinzendorfstraße 5, 01069 Dresden"
df.loc[id, "extrMultipleDaysDetails"] = """Mi – Fr 16 – 19 Uhr Sa & So und Feiertag 12 -18 Uhr"""


In [ ]:
id = max(df.index) + 1
print(id)
flt = df["Event_description"].str.contains("Eine Ausstellung zeitgenössischer Kunst zu den Auswirkungen rassistischer", na=False)
assert len(df[flt])>0
df = df[~flt]
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationName"]="Robotron-Kantine in Dresden"
df.loc[id, "extrStreetHouseNumber"]="Zinzendorfstraße 5"
df.loc[id, "extrLocationDetails"]="Zinzendorfstraße 5, 01069 Dresden"
df.loc[id, "extrMultipleDaysDetails"] = """Mi – Fr 16 – 19 Uhr Sa & So und Feiertag 12 -18 Uhr"""


In [ ]:
id = max(df.index) + 1
print(id)
flt = df["Event_description"].str.contains('Unter dem Motto "Über die Natur" findet zum vierten Mal ', na=False)
assert len(df[flt])>0
df = df[~flt]
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationName"]="Schloss Pillnitz"
df.loc[id, "extrLocationDetails"]="August-Böckstiegel-Straße 2, 01326 Dresden"
df.loc[id, "extrStreetHouseNumber"]="August-Böckstiegel-Straße 2"
df.loc[id, "extrMultipleDaysDetails"] = """täglich 10-17 Uhr, Montag geschlossen"""


In [ ]:
id = max(df.index) + 1
print(id)
flt = df["Event_description"].str.contains('Mikroschnitzereien auf Streichhölzern oder in Walnussschalen, Modelle und Puppenspielzeug', na=False)
assert len(df[flt])>0
df = df[~flt]
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-20"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationDetails"]="Köpckestraße 1, 01097 Dresden"
df.loc[id, "extrStreetHouseNumber"]="Köpckestraße 1"
df.loc[id, "extrMultipleDaysDetails"] = """täglich 10-18 Uhr, Montag geschlossen"""


In [ ]:
id = max(df.index) + 1
print(id)
flt = df["Event_description"].str.contains('Dem langjährigen Professor für Monumentalmalerei widmet sich die neue Sonderausstellung', na=False)
assert len(df[flt])>0
df = df[~flt]
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationName"]="Schloss Pillnitz"
df.loc[id, "extrLocationDetails"]="August-Böckstiegel-Straße 2, 01326 Dresden"
df.loc[id, "extrStreetHouseNumber"]="August-Böckstiegel-Straße 2"
df.loc[id, "extrMultipleDaysDetails"] = """Dienstag bis Sonntag 10 bis 17 Uhr"""


In [ ]:
id = max(df.index) + 1
print(id)
flt = df["Event_description"].str.contains('Tonband- und Kassettengeräte kam man', na=False)
assert len(df[flt])>0
df = df[~flt]
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-19"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationName"]=" Städtische Galerie Dresden"
df.loc[id, "extrMultipleDaysDetails"] = """Eröffnung: DO 19.09. 19.00 Uhr DI bis SO 10.00 bis 18.00 Uhr FR 10.00 bis 19.00 Uhr"""


In [ ]:
id = max(df.index) + 1
print(id)
flt = df["Event_description"].str.contains('100 Jahre KUNSTAUSSTELLUNG KÜHL in Dresden.', na=False)
assert len(df[flt])>0
df = df[~flt]
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationName"]=" Kunstausstellung Kühl Dresden"
df.loc[id, "extrStreetHouseNumber"]="Nordstraße 5"
df.loc[id, "extrMultipleDaysDetails"] = """Mi bis Fr 11 –19 + Sa 11 –16 Uhr"""


In [ ]:
id = max(df.index) + 1
print(id)
flt = df["Event_description"].str.contains('Auf die Elbe, fertig, los!', na=False)
assert len(df[flt])>0
df = df[~flt]
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationName"]="Blaues Wunder"
df.loc[id, "extrStartTime"] = "11"
df.loc[id, "extrEndTime"] = "15"

In [ ]:
id = max(df.index) + 1
print(id)
flt = df["Event_description"].str.contains('Das Highlight für Kinder ist die Parkeisenbahn.', na=False)
assert len(df[flt])>0
df = df[~flt]
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-26"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationName"]="Parkeisenbahn Dresden"
df.loc[id, "extrLocationDetails"]="01219 Dresden-Altstadt"
df.loc[id, "extrMultipleDaysDetails"] = """Mittwoch bis Freitag: 13:00 bis 18:00 Uhr Samstag, Sonntag, Feiertag: 10:30 bis 18:00 Uhr"""

In [ ]:
id = max(df.index) + 1
print(id)
flt = df["Event_description"].str.contains('Süße, halb tote Katze verschwunden! Ihre Spur', na=False)
assert len(df[flt])>0
df = df[~flt]
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationName"]="Technische Sammlungen Dresden"
df.loc[id, "extrLocationDetails"]="01277 Dresden Junghannsstraße 1-3"
df.loc[id, "extrStreetHouseNumber"]="Junghannsstraße 1-3"
df.loc[id, "extrMultipleDaysDetails"] = """DI bis FR 9.00 bis 17.00 Uhr SA/SO 10.00 bis 18.00 Uhr"""

In [ ]:
id = max(df.index) + 1
print(id)
flt = df["Event_description"].str.contains('prägenden Periode in der Geschichte des DHMD und stellt sie in den Kontext der wohl einschneidendsten Umbrüche der', na=False)
assert len(df[flt])>0
df = df[~flt]
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationName"]="Hygienemuseum"
df.loc[id, "extrLocationDetails"]="01069 Dresden Lingnerplatz 1"
df.loc[id, "extrStreetHouseNumber"]="Lingnerplatz 1"
df.loc[id, "extrMultipleDaysDetails"] = """Dienstag bis Sonntag, Feiertage: 10 bis 18 Uhr"""

In [ ]:
""" id = max(df.index) + 1
print(id)
flt = df["Event_description"].str.contains('prägenden Periode in der Geschichte des DHMD und stellt sie in den Kontext der wohl einschneidendsten Umbrüche der', na=False)
assert len(df[flt])>0
df = df[~flt]
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationName"]="Hygienemuseum"
df.loc[id, "extrLocationDetails"]="01069 Dresden Lingnerplatz 1"
df.loc[id, "extrStreetHouseNumber"]="Lingnerplatz 1"
df.loc[id, "extrMultipleDaysDetails"] = "Dienstag bis Sonntag, Feiertage: 10 bis 18 Uhr" """

In [ ]:
""" id = max(df.index) + 1
print(id)
flt = df["Event_description"].str.contains('Mit dem „Fast Forward“-Knopf analoger Tonband- und Kassettengeräte kam man in beschleunigter', na=False)
assert len(df[flt])>0
df = df[~flt]
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-19"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationDetails"]="01067 Dresden Wilsdruffer Str. 2"
df.loc[id, "extrStreetHouseNumber"]="Wilsdruffer Str. 2"
df.loc[id, "extrMultipleDaysDetails"] = "Eröffnung: DO 19.09. 19.00 Uhr DI bis SO 10.00 bis 18.00 Uhr FR 10.00 bis 19.00 Uhr" """

In [ ]:
""" id = max(df.index) + 1
print(id)
flt = df["Event_description"].str.contains('Dem langjährigen Professor für Monumentalmalerei widmet sich die neue Sonderausstellung im Pillnitzer Schlossmuseum', na=False)
assert len(df[flt])>0
df = df[~flt]
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationDetails"]="01326 Dresden-Pillnitz August-Böckstiegel-Straße 2"
df.loc[id, "extrStreetHouseNumber"]="August-Böckstiegel-Straße 2"
df.loc[id, "extrMultipleDaysDetails"] = "Dienstag bis Sonntag 10 bis 17 Uhr" """

In [ ]:
id = max(df.index) + 1
print(id)
flt = df["Event_URL"].str.contains('https://sachsen-net.com/veranstaltungen/heute/Kinderbiennale-Planet-Utopia-Dresden-114311.html', na=False)
assert len(df[flt])>0
df = df[~flt]
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationDetails"]="01097 Dresden Palaisplatz 11"
df.loc[id, "extrStreetHouseNumber"]="Palaisplatz 11"
df.loc[id, "extrMultipleDaysDetails"] = "Diensatg bis Sonntag 10 bis 18 Uhr"

In [ ]:
id = max(df.index) + 1
print(id)
flt = df["Event_URL"].str.contains('https://sachsen-net.com/veranstaltungen/heute/Wiederentdeckt-Dresdner-Malerinnen-Der-Romantik-Dresden-116639.html', na=False)
assert len(df[flt])>0
df = df[~flt]
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationDetails"]="01097 Dresden Hauptstr. 13"
df.loc[id, "extrMultipleDaysDetails"] = "MI bis FR 10.00 bis 17.00 Uhr SA/SO 12.00 bis 17.00 Uhr"

In [ ]:
id = max(df.index) + 1
print(id)
flt = df["Event_URL"].str.contains('https://sachsen-net.com/veranstaltungen/heute/Von-Wanderlust-Und-Reisefrust', na=False)
assert len(df[flt])>0
df = df[~flt]
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-27"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationDetails"]="01099 Dresden Nordstraße 28"
df.loc[id, "extrMultipleDaysDetails"] = "MI bis SO 12.00 bis 17.00 Uhr"

In [ ]:
id = max(df.index) + 1
print(id)
flt = df["Event_URL"].str.contains('https://sachsen-net.com/veranstaltungen/heute/Ausstellung-Home-Sweet-Home-Danny-Hermann-Radebeul-117962.html', na=False)
assert len(df[flt])>0
df = df[~flt]
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-07"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrMultipleDaysDetails"] = "Samstag & Sonntag 14-18 Uhr"

In [ ]:
id = max(df.index) + 1
print(id)
flt = df["Event_URL"].str.contains('https://sachsen-net.com/veranstaltungen/heute/Sonderausstellung-Andreas-Braeunsdorf-Malerei-Dresden-116638.html', na=False)
assert len(df[flt])>0
df = df[~flt]
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-07"
df.loc[id, "extrEndDate"] = "2024-09-29"

In [ ]:
id = max(df.index) + 1
print(id)
flt = df["Event_URL"].str.contains('https://sachsen-net.com/veranstaltungen/heute/Udo-Zimmermann-Ein-Moderner-Romantiker-Dresden-114359.html', na=False)
assert len(df[flt])>0
df = df[~flt]
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-09-29"
df.loc[id, "extrMultipleDaysDetails"] = "MI bis SO 12.00 bis 17.00 Uhr"

In [ ]:
id = max(df.index) + 1
print(id)
flt1 = df["Event_URL"].str.contains('https://sachsen-net.com/veranstaltungen/heute/Trickfilm-Plakat-Kunst-Arbeiten-Fuer-Den-Ddr-Filmverleih-113199.html', na=False)
assert len(df[flt])>0
df = df[~flt]
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-31"

In [ ]:
id = max(df.index) + 1
print(id)
flt = df["Event_URL"].str.contains('https://sachsen-net.com/veranstaltungen/heute/Panorama-Amazonien-Im-Panometer-Dresden-Dresden-113732.html', na=False)
assert len(df[flt])>0
df = df[~flt]
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-31"

In [ ]:
id = max(df.index) + 1
print(id)
flt = df["Event_URL"].str.contains('https://sachsen-net.com/veranstaltungen/heute/Zeitfenster-Spurensuche-Zur-Industriegeschichte-Im-Ernemannbau-Dresden-113361.html', na=False)
assert len(df[flt])>0
df = df[~flt]
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-20"

In [ ]:
id = max(df.index) + 1
print(id)
flt = df["Event_URL"].str.contains('https://sachsen-net.com/veranstaltungen/heute/Physics-Of-Life-Physik-Des-Lebens-Dresden-109648.html', na=False)
assert len(df[flt])>0
df = df[~flt]
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-27"

In [ ]:
id = max(df.index) + 1
print(id)
flt = df["Event_URL"].str.contains('https://sachsen-net.com/veranstaltungen/heute/Trickfilm-Plakat-Kunst-Arbeiten-Fuer-Den-Ddr-Filmverleih-113199.html', na=False)
assert len(df[flt])>0
df = df[~flt]
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-31"

In [ ]:
id = max(df.index) + 1
print(id)
flt = df["Event_URL"].str.contains('https://sachsen-net.com/veranstaltungen/heute/Dresdner-Herbstvolksfest-Der-Freizeitpark-Dresden-119283.html', na=False)
assert len(df[flt])>0
df = df[~flt]
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-10-03"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrMultipleDaysDetails"] = """ Montag bis Donnerstag 15:00 - 22:00 Uhr
Freitag 15:00 - 23:00 Uhr
Samstag 14:00 - 23:00 Uhr
Sonntag 14:00 - 22:00 Uhr
SCHLIESSTAGE 21.10., 22.10., 28.10., 29.10.2024"""
df.loc[id, "extrLocationName"]="Volksfestgelände Pieschener Allee"
df.loc[id, "extrLocationDetails"]="01067 Dresden Pieschener Allee 1"

In [503]:
df.loc[:, "count_Event_description"]= df.Event_description.map(df.Event_description.value_counts())

In [ ]:
df_urls = df.query("count_Event_description>1 & extrMultipleDaysDetails!='nan' & extrLocationName!='nan' & extrStartDate=='nan' ")[["Event_URL"]].drop_duplicates()
print(len(df_urls))
df_urls

In [ ]:
len(df)

In [ ]:
df.query("count_Event_description>20")["Event_URL"].drop_duplicates()

In [507]:
flt = (df.extrStartTime=="nan") & (df.isOnMultipleDays==False)
#df.loc[flt]

In [ ]:
df.query("isOnMultipleDays==False & OpenAIResponse_MultipleDaysDetails!='nan'")

In [ ]:
len(df[df.OpenAIResponse_StartDate.str.contains("2023") & (df.isOnMultipleDays==False)])

# final checks and keep only suitable observations

In [ ]:
len(df)

In [511]:
extracted_columns = []
for col in df.columns:
    if col.startswith("extr"):
        extracted_columns.append(col)
    
extracted_columns.append("isOnMultipleDays")

In [ ]:
for col in extracted_columns:
    print(f"{col=} has missing values:")
    flt = df[col]=="nan"
    print(len(df[flt]))


In [ ]:
flt = (df.isOnMultipleDays==False ) & (df.extrStartDate!="nan") & (df.extrStartTime!="nan") &\
    ( (df.extrLocationName!="nan") | (df.extrLocationDetails!="nan") | (df.extrStreetHouseNumber!="nan")) 
len(df[flt])

In [ ]:
df.loc[flt, extracted_columns]

## save

In [515]:
exclude_columns= ['Date (GMT)', 'Job Link']

subset_columns = [col for col in df.columns if col not in exclude_columns]

df = df.drop_duplicates(subset=subset_columns)

In [ ]:
len(df)

In [517]:
df = df.dropna(how="all", axis=1)

In [ ]:
# df.to_excel(f"{DATA_FOLDER}df_browse_ai {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.xlsx")

In [521]:
df.to_csv(f"{DATA_FOLDER}df_browse_ai {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.csv")

In [522]:
df.to_pickle(f"{DATA_FOLDER}df_browse_ai {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.pkl")